In [ ]:
# 1. Explore contours

from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
from pykrige.uk import UniversalKriging
import cmocean
import pandas as pd


plt.clf();                
cmap = cmocean.cm.balance;#sns.cubehelix_palette(as_cmap=True)
fig, ax = plt.subplots(figsize=(8, 4))

df = pd.read_csv('2018_spr.csv',sep=';',encoding='cp1251',header = 'infer')
df[df==""] = np.NaN
df = df.ffill()
print(df.shape)

df = df[(df['долготаdd'] < 33.6) & (df['долготаdd'] > 33.4)]
df = df[['Z, m','широтаdd','S, psu']]
print(df.shape)

df['широтаdd'] = 111.6*(df['широтаdd']-69.5)
df = df.iloc[::3, :]
print(df.shape)
data = df.to_numpy()

bottom = np.genfromtxt("bottom.txt", delimiter=" ");  
bottom[:,0] = [111.6*(a-69.5) for a in bottom[:,0]]
contarr = [34.95,34.97,34.99]
gridx = np.arange(0.0, 1000.0, 4)
gridy = np.arange(0.0, 350.0, 3)
UK = UniversalKriging(data[:, 1], data[:, 0], data[:, 2], variogram_model='linear',
                              drift_terms=['regional_linear'])
z, ss = UK.execute('grid', gridx, gridy)
plt.scatter(data[:, 1], data[:, 0],s=2)

rr, cc = np.meshgrid(gridx, gridy)
CS = ax.contour(rr, cc, z,100,levels=contarr,c=z, 
                 linewidths=0.5, cmap=cmocean.cm.balance,zorder = 2)
cl = ax.clabel(CS, CS.levels[::1],inline=False, fmt='%.2f',colors='black', fontsize=6)

points = plt.scatter(bottom[:,0],bottom[:,1],s=1) #,c=data[:,2], s=10, cmap=cmap,zorder=2,vmin=valmin, vmax=valmax)
plt.xlabel("Расстояние, км");
plt.ylabel("Глубина, м");
plt.xticks([w*100 for w in range(11)], [str(w*100) for w in range(11)]);
plt.xlim(0,1050);
plt.ylim(0,350)
plt.grid (True , linestyle= '-', color='0.75',zorder=0)

ax.set_ylim(ax.get_ylim()[::-1])
plt.savefig("main_contours.png",bbox_inches='tight',pad_inches=0.05, dpi=300);
plt.close();



In [ ]:
# 2. Plot water mass polygon (above bottom) pb and all the considered contours from CS.allsegs[1]

from shapely.geometry import Polygon

fig, ax = plt.subplots(figsize=(8, 4))
for i in range(2):    
    dat0= CS.allsegs[1][i]
    plt.plot(dat0[:,0],dat0[:,1])

pb = Polygon([(i[0], i[1]) for i in zip(bottom[:,0],bottom[:,1])])
plt.plot(*pb.exterior.xy)
ax.set_ylim(ax.get_ylim()[::-1])

print(len(CS.allsegs[1]))

In [ ]:
# 3. All together. 

from matplotlib import pyplot as plt
import numpy as np
from shapely.geometry import Polygon
%matplotlib inline
from matplotlib.path import Path
from matplotlib.patches import PathPatch

from matplotlib.patches import Polygon as plotpoly
from matplotlib.collections import PatchCollection
import cmocean

plt.clf();                
cmap = cmocean.cm.balance;
fig, ax = plt.subplots(figsize=(6, 4))
patches = []

total = 0
contarr = [34.95,34.97,34.99]

rr, cc = np.meshgrid(gridx, gridy)
CSF = plt.contourf(rr, cc,z)

plt.xlabel("Расстояние, км");
plt.ylabel("Глубина, м");
plt.xticks([w*100 for w in range(11)], [str(w*100) for w in range(11)]);
plt.xlim(0,1000);
plt.ylim(0,350)
plt.grid (linestyle= '-', color='0.75',zorder=0)

polygon = plotpoly(bottom, True)
patches.append(polygon)
p = PatchCollection(patches)               
p.zorder = 150                   
p.set(color = "lightgrey")
ax.add_collection(p)

collection = CS.collections[1]
p = collection.get_paths()[1]
v = p.vertices
x = v[0:,0]
y = v[0:,1]
li = [(i[0], i[1]) for i in zip(x,y)]
print(len(li))

p = collection.get_paths()[0]
v = p.vertices
x = v[:,0]
y = v[:,1]
l2 = [(i[0], i[1]) for i in zip(x,y)]
print(len(l2))
for ii in range(len(l2)):
    li.append(l2[ii])
li.append((x[len(l2)-1],350))
li.append((380,350))
p_p = Polygon(li)
pp_p = p_p.intersection(pb)
p = pp_p
total = p.area*0.001
plt.fill(*p.exterior.xy,"yellow");

plt.scatter(data[:, 1], data[:, 0],s=1,c="black",zorder = 50)

dddd = np.array(li)
plt.plot(dddd[:,0],dddd[:,1],color="red",zorder = 100)

ax.text(10, 345, 'Площадь ≈ '+str(float("{0:.1f}".format(total))) +' км²' , style='italic', color='red', fontsize=10,zorder = 150)

ax.text(30, 300, 'Мурманская\nвозвышенность', style='italic', color='green', fontsize=7,zorder = 150)
ax.text(270, 325, 'Финмаркенская\n        банка', style='italic', color='green', fontsize=7,zorder = 150)

ax.text(400, 345, 'Демидовский\n     жёлоб', style='italic', color='green', fontsize=7,zorder = 150)
ax.text(560, 300, 'Центральная\n     банка', style='italic', color='green', fontsize=7,zorder = 150)
ax.text(700, 345, 'Жёлоб\nПерсея', style='italic', color='green', fontsize=7,zorder = 150)
ax.text(800, 300, 'Возвышенность\n     Персея', style='italic', color='green', fontsize=7,zorder = 150)

ax.text(0, -2, "69°30\'", style='italic', color='black', fontsize=12,zorder = 20)

ax.text(210, 150, "34.97", style='italic', color='red', fontsize=10,zorder = 100)

ax.set_ylim(ax.get_ylim()[::-1])
plt.savefig("KM_2018_apr.jpg",bbox_inches='tight',pad_inches=0.1, dpi=600);
plt.close();